In [2]:
# Hello there.   This is a do-over of the floating preferred analyzer

# Time to develop clean modules. Let's start with reading in a list of prefs
# The input data consists of three separate tables:
# a) A listing of tickers and their configurations
# b) A list of ratings for each company (equivalently, of the ticker)
# c) The current interest rate environment, which has only the 3-month rate and the prime rate



In [3]:
# Imports go here
import pandas as pd
import requests_cache
# local ones
import prefcode as pc


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# Set up cache
session_cached = requests_cache.CachedSession('yfinance.cache',expire_after=3600)
session_uncached = requests_cache.CachedSession('yfinance2.cache',expire_after=30)


In [6]:
# Read the prefs database

print("Setting up databases")
df, interest_db = pc.setup_databases()
interest_db = {
    'tbill': 2.65, 'goc5': 2.75, 'prime': 3.2}

print(interest_db)

print("Fetching price data")
df = pc.fetch_prices(df, session_uncached, fetch=True)
df.head(10).sort_values(by='Price', ascending=True)

Setting up databases
{'tbill': 2.65, 'goc5': 2.75, 'prime': 3.2}
Fetching price data


,Ticker,Spread,Type,Mult,Rating,Price
2,BAM.PR.C,NaN,P,0.7,P2L,12.81
4,BAM.PR.K,NaN,P,0.7,P2L,12.99
1,BAM.PR.B,NaN,P,0.7,P2L,13.00
8,BCE.PR.N,209.0,T,NaN,P3I,16.75
0,ALA.PR.B,266.0,T,NaN,P3L,17.50
9,BCE.PR.S,0.0,P,1.0,P3I,17.90
7,BCE.PR.J,0.0,P,1.0,P3I,18.06
6,BCE.PR.D,0.0,P,1.0,P3I,18.24
5,BCE.PR.B,0.0,P,1.0,P3I,18.50
3,BAM.PR.E,NaN,P,1.0,P2L,18.68


In [7]:
print("Updating dividends and current yield")
df = pc.update_div_and_yield(df, interest_db,price_column="Price")

Updating dividends and current yield


In [8]:
print("Updating market spread for later calculation")
df2 = pc.update_market_spread(df, interest_db)

print("Dropping prime-related preferreds. Maybe another day")
tdf = df[df['Type'] == 'T'].copy()

print("Calculating scenarios for T-bill rate")
scenarios = {"265":  [2.65,  0.50],
             "315":  [3.15,  0.30],
             "340":  [3.40,  0.15],
             "365":  [3.65,  0.05]}

pc.update_expected_yield(tdf, scenarios)

Updating market spread for later calculation
Dropping prime-related preferreds. Maybe another day
Calculating scenarios for T-bill rate


In [9]:
print("Here are the highest yield items")
tdf.sort_values(by='ExpYield', ascending=False).head(15)

Here are the highest yield items


,Ticker,Spread,Type,Mult,Rating,Price,AnnualDiv,CurYieldPct,MSpread,265_Yield,315_Yield,340_Yield,365_Yield,ExpYield
20,FN.PR.B,207.0,T,NaN,P3I,14.20,1.1800,8.3099,5.6599,8.3094,12.6259,14.6055,16.4788,10.957235
30,TRP.PR.H,128.0,T,NaN,P2L,14.00,0.9825,7.0179,4.3679,7.0172,12.2429,14.6035,16.8168,10.212835
14,CVE.PR.B,173.0,T,NaN,P2L,15.00,1.0950,7.3000,4.6500,7.3000,11.5735,13.5112,15.3321,9.915335
27,TA.PR.E,203.0,T,NaN,P3L,15.75,1.1700,7.4286,4.7786,7.4282,11.1319,12.8139,14.3962,9.695565
31,TRP.PR.I,154.0,T,NaN,P2L,15.05,1.0475,6.9601,4.3101,6.9606,11.3917,13.3917,15.2661,9.669870
24,PWF.PR.Q,160.0,T,NaN,P2H,15.15,1.0625,7.0132,4.3632,7.0132,11.3400,13.2944,15.1269,9.659105
17,FFH.PR.F,216.0,T,NaN,P3H,16.01,1.2025,7.5109,4.8609,7.5113,11.0161,12.6093,14.1090,9.657325
25,SLF.PR.J,141.0,T,NaN,P2H,15.15,1.0150,6.6997,4.0497,6.6992,11.2145,13.2448,15.1433,9.457835
22,MFC.PR.P,141.0,T,NaN,P2I,15.20,1.0150,6.6776,4.0276,6.6781,11.1694,13.1883,15.0757,9.421900
15,EMA.PR.B,184.0,T,NaN,P3H,15.90,1.1225,7.0597,4.4097,7.0604,10.8458,12.5567,14.1614,9.375515


In [10]:
print("And uniquified by issuer")
# Uniquify by parent
tdf['Parent'] = [x.split('.')[0] for x in tdf['Ticker']]
#tdf
uniq_parent = pc.summarize_best_by_column(tdf,'ExpYield','Parent')

uniq_parent = uniq_parent.reindex(columns=["Ticker", "Rating", 
                                           "Spread","CurYieldPct","ExpYield"])
# make it a bit more readable

uniq_parent['CurYield'] = [round(x,3) for x in uniq_parent['CurYieldPct']]
uniq_parent['ExpYield'] = [round(x,3) for x in uniq_parent['ExpYield']]
uniq_parent.drop(columns=['CurYieldPct'], errors='ignore', inplace=True)
uniq_parent.sort_values(by='ExpYield', ascending=False).head(15)


And uniquified by issuer


,Ticker,Rating,Spread,ExpYield,CurYield
20,FN.PR.B,P3I,207.0,10.957,8.310
30,TRP.PR.H,P2L,128.0,10.213,7.018
14,CVE.PR.B,P2L,173.0,9.915,7.300
27,TA.PR.E,P3L,203.0,9.696,7.429
24,PWF.PR.Q,P2H,160.0,9.659,7.013
17,FFH.PR.F,P3H,216.0,9.657,7.511
25,SLF.PR.J,P2H,141.0,9.458,6.700
22,MFC.PR.P,P2I,141.0,9.422,6.678
15,EMA.PR.B,P3H,184.0,9.376,7.060
21,FTS.PR.I,P3H,145.0,9.244,6.613


In [11]:
best_scn_df = pc.summarize_best_by_column(tdf, 'ExpYield')
best_scn_df = best_scn_df.reindex(columns=["Ticker", "Rating", "Spread","ExpYield", "MSpread"])

print("And here are the best by rating level")
best_scn_df.sort_values(by=['Rating'])

And here are the best by rating level


,Ticker,Rating,Spread,ExpYield,MSpread
24,PWF.PR.Q,P2H,160.0,9.659105,4.3632
22,MFC.PR.P,P2I,141.0,9.421900,4.0276
30,TRP.PR.H,P2L,128.0,10.212835,4.3679
17,FFH.PR.F,P3H,216.0,9.657325,4.8609
20,FN.PR.B,P3I,207.0,10.957235,5.6599
27,TA.PR.E,P3L,203.0,9.695565,4.7786


In [12]:
# Now, check market spread and flag if below average for the group
# That could result in a re-rating
#mdf = pc.calculate_avg_per_rating(tdf, column='MSpread')
#mdf